# CasualMarket MCP 工具測試手冊


In [1]:
from agents.mcp import MCPServerStdio
import json

# 修復後的參數設定 - 這是正確的配置
params = {
    "command": "uv",
    "args": [
        "run",
        "--directory",
        "/Users/sacahan/Documents/workspace/CasualMarket",
        "python",
        "-m",
        "src.main",
    ],
    "env": {"MARKET_MCP_LOG_FILE": "./logs/mcp.log", "MARKET_MCP_RATE_LIMIT_INTERVAL": "5.0"},
}

## 1. 交易相關工具測試

## 0. 快取測試（使用單一 Server 實例）

⚠️ **重要**：為了測試快取效果，下面的測試使用**相同的 server 實例**執行多次查詢。
這樣快取才能在同一進程中生效，您可以看到日誌中的 `[快取命中]` 記錄。


In [2]:
import time

print("=" * 80)
print("快取演示：同一 Server 實例中的多次查詢")
print("=" * 80)

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    # 第一次查詢 - 應該命中 API
    print("\n【查詢 1】get_company_profile (3231) - 第一次")
    start_time_1 = time.time()
    result1 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time1 = (time.time() - start_time_1) * 1000
    print(f"✓ 耗時: {time1:.2f}ms")
    data1 = json.loads(result1.content[0].text)
    print(f"✓ 資料欄位: {list(data1.get('data', {}).keys())[:5]}...")  # 只顯示前5個欄位

    time.sleep(0.3)

    # 第二次查詢相同資料 - 應該從快取讀取（快很多）
    print("\n【查詢 2】get_company_profile (3231) - 第二次（應該從快取讀取）")
    start_time_2 = time.time()
    result2 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time2 = (time.time() - start_time_2) * 1000
    print(f"✓ 耗時: {time2:.2f}ms")
    
    # 效能對比
    print("\n" + "=" * 80)
    print("【效能對比】")
    print("=" * 80)
    print(f"第一次 (API):     {time1:.2f}ms")
    print(f"第二次 (快取):    {time2:.2f}ms")
    speedup = time1 / time2 if time2 > 0 else float('inf')
    print(f"加速比:           {speedup:.1f}x")
    
    if time2 < time1 * 0.5:
        print("✅ 快取正常運作！")
    else:
        print("⚠️  快取可能未生效")

    time.sleep(0.3)

    # 第三次查詢不同資料 - 應該命中 API
    print("\n【查詢 3】get_company_profile (2330) - 不同股票，應該命中 API")
    start_time_3 = time.time()
    result3 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    time3 = (time.time() - start_time_3) * 1000
    print(f"✓ 耗時: {time3:.2f}ms")

    time.sleep(0.3)

    # 第四次查詢相同資料 - 應該從快取讀取
    print("\n【查詢 4】get_company_profile (2330) - 第二次（應該從快取讀取）")
    start_time_4 = time.time()
    result4 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    time4 = (time.time() - start_time_4) * 1000
    print(f"✓ 耗時: {time4:.2f}ms")

print("\n✓ 測試完成！請查看日誌檔案查看詳細的 [快取命中] 和 [已快取] 記錄")
print(f"  日誌位置: ./logs/mcp.log")


快取演示：同一 Server 實例中的多次查詢

【查詢 1】get_company_profile (3231) - 第一次
✓ 耗時: 8817.52ms
✓ 資料欄位: ['出表日期', '公司代號', '公司名稱', '公司簡稱', '外國企業註冊地國']...

【查詢 2】get_company_profile (3231) - 第二次（應該從快取讀取）
✓ 耗時: 18.43ms

【效能對比】
第一次 (API):     8817.52ms
第二次 (快取):    18.43ms
加速比:           478.4x
✅ 快取正常運作！

【查詢 3】get_company_profile (2330) - 不同股票，應該命中 API
✓ 耗時: 16.59ms

【查詢 4】get_company_profile (2330) - 第二次（應該從快取讀取）
✓ 耗時: 13.68ms

✓ 測試完成！請查看日誌檔案查看詳細的 [快取命中] 和 [已快取] 記錄
  日誌位置: ./logs/mcp.log


In [ ]:
print("=" * 80)
print("🔧 修復後的快取測試")
print("=" * 80)

async with MCPServerStdio(params=params, client_session_timeout_seconds=120) as server:
    print("\n📊 清除舊日誌並開始新測試...")
    
    # 清除日誌檔案以便觀察新的日誌
    import os
    log_path = "./logs/mcp.log"
    if os.path.exists(log_path):
        # 備份舊日誌
        import shutil
        backup_path = f"{log_path}.backup"
        shutil.copy2(log_path, backup_path)
        print(f"✓ 已備份舊日誌到 {backup_path}")
        
        # 清空日誌檔案
        with open(log_path, 'w') as f:
            f.write("")
        print(f"✓ 已清空 {log_path}")
    
    time.sleep(1)  # 等待日誌系統重新初始化
    
    # === 測試 1: 相同公司的重複查詢 ===
    print("\n" + "="*60)
    print("📋 測試 1: 相同公司 (3231) 的重複查詢")
    print("="*60)
    
    # 第一次查詢 3231
    print("■ 查詢 1: get_company_profile(3231) - 第 1 次 (應命中 API)")
    start = time.time()
    result1 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time1 = (time.time() - start) * 1000
    print(f"  ✓ 耗時: {time1:.2f}ms")
    
    time.sleep(0.5)
    
    # 第二次查詢 3231（應該從快取讀取）
    print("■ 查詢 2: get_company_profile(3231) - 第 2 次 (應從快取讀取)")
    start = time.time()
    result2 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time2 = (time.time() - start) * 1000
    print(f"  ✓ 耗時: {time2:.2f}ms")
    
    # 分析結果
    speedup1 = time1 / time2 if time2 > 0 else float('inf')
    print(f"\n📈 效能對比:")
    print(f"  第 1 次: {time1:.2f}ms")
    print(f"  第 2 次: {time2:.2f}ms") 
    print(f"  加速比: {speedup1:.1f}x")
    
    if time2 < time1 * 0.5:
        print("  ✅ 快取生效！")
    else:
        print("  ⚠️  快取可能未生效")
    
    time.sleep(0.5)
    
    # === 測試 2: 不同公司的查詢 ===
    print("\n" + "="*60)
    print("📋 測試 2: 不同公司 (2330) 的查詢")
    print("="*60)
    
    # 第一次查詢 2330
    print("■ 查詢 3: get_company_profile(2330) - 第 1 次 (應命中 API)")
    start = time.time()
    result3 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    time3 = (time.time() - start) * 1000
    print(f"  ✓ 耗時: {time3:.2f}ms")
    
    time.sleep(0.5)
    
    # 第二次查詢 2330（應該從快取讀取）
    print("■ 查詢 4: get_company_profile(2330) - 第 2 次 (應從快取讀取)")
    start = time.time()
    result4 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    time4 = (time.time() - start) * 1000
    print(f"  ✓ 耗時: {time4:.2f}ms")
    
    # 分析結果
    speedup2 = time3 / time4 if time4 > 0 else float('inf')
    print(f"\n📈 效能對比:")
    print(f"  第 1 次: {time3:.2f}ms")
    print(f"  第 2 次: {time4:.2f}ms")
    print(f"  加速比: {speedup2:.1f}x")
    
    if time4 < time3 * 0.5:
        print("  ✅ 快取生效！")
    else:
        print("  ⚠️  快取可能未生效")
    
    time.sleep(0.5)
    
    # === 測試 3: 再次查詢第一個公司（驗證快取持續有效）===
    print("\n" + "="*60)
    print("📋 測試 3: 再次查詢 3231（驗證快取持續有效）")
    print("="*60)
    
    print("■ 查詢 5: get_company_profile(3231) - 第 3 次 (應從快取讀取)")
    start = time.time()
    result5 = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time5 = (time.time() - start) * 1000
    print(f"  ✓ 耗時: {time5:.2f}ms")
    
    speedup3 = time1 / time5 if time5 > 0 else float('inf')
    print(f"  與第 1 次對比: {speedup3:.1f}x 加速")
    
    if time5 < time1 * 0.5:
        print("  ✅ 快取持續有效！")
    else:
        print("  ⚠️  快取可能已失效")

print("\n" + "=" * 80)
print("🔍 測試完成！立即檢查日誌檔案中的快取記錄")
print("=" * 80)
print("執行下一個 cell（快取配置檢查工具）來查看詳細統計")

### 1.1 取得台灣股票即時價格

## 1.1.1 單一 Server 實例測試（推薦用於快取驗證）

使用單一 server 實例測試多個股票查詢，觀察快取效果。


In [ ]:
import time

# 單一 server 實例，執行多個查詢
async with MCPServerStdio(params=params, client_session_timeout_seconds=120) as server:
    print("=" * 80)
    print("【單一 Server 實例多查詢測試】")
    print("=" * 80)
    
    test_symbols = ["2317", "台積電", "0050"]
    
    for symbol in test_symbols:
        print(f"\n■ 查詢: {symbol} (第一次 - 命中 API)")
        start = time.time()
        result = await server.session.call_tool(
            "get_taiwan_stock_price",
            {"symbol": symbol}
        )
        elapsed = (time.time() - start) * 1000
        print(f"  耗時: {elapsed:.2f}ms")
        
        time.sleep(0.2)
        
        print(f"■ 查詢: {symbol} (第二次 - 應從快取讀取)")
        start = time.time()
        result = await server.session.call_tool(
            "get_taiwan_stock_price",
            {"symbol": symbol}
        )
        elapsed = (time.time() - start) * 1000
        print(f"  耗時: {elapsed:.2f}ms ⚡ (應該快速)")
        
        time.sleep(0.3)

print("\n✓ 測試完成！查看 logs/mcp.log 中的 [快取命中] 記錄")



---

### 📚 對比示例：不同 Server 實例 vs 相同 Server 實例


In [ ]:
print("=" * 80)
print("❌ 不推薦：使用不同的 Server 實例")
print("=" * 80)
print("每次呼叫都建立新 server → 新進程 → 快取無法共享\n")

times_separate = []
for i in range(2):
    print(f"查詢 {i+1}: 使用新的 server 實例")
    start = time.time()
    async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
        result = await server.session.call_tool(
            "get_company_profile",
            {"symbol": "3231"}
        )
    elapsed = (time.time() - start) * 1000
    times_separate.append(elapsed)
    print(f"  耗時: {elapsed:.2f}ms (都是 API 呼叫，沒有快取效果)\n")

print("\n" + "=" * 80)
print("✅ 推薦：使用相同的 Server 實例")
print("=" * 80)
print("同一個 server → 同一進程 → 快取有效\n")

times_same = []
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    for i in range(2):
        print(f"查詢 {i+1}: 使用相同的 server 實例")
        start = time.time()
        result = await server.session.call_tool(
            "get_company_profile",
            {"symbol": "3231"}
        )
        elapsed = (time.time() - start) * 1000
        times_same.append(elapsed)
        if i == 0:
            print(f"  耗時: {elapsed:.2f}ms (第一次，命中 API)")
        else:
            print(f"  耗時: {elapsed:.2f}ms (第二次，從快取讀取 ⚡)")
        time.sleep(0.2)

print("\n" + "=" * 80)
print("【效能對比】")
print("=" * 80)
print(f"不同 server 實例:")
print(f"  查詢 1: {times_separate[0]:.2f}ms")
print(f"  查詢 2: {times_separate[1]:.2f}ms")
print(f"  → 都是 API 呼叫，時間相近")

print(f"\n相同 server 實例:")
print(f"  查詢 1: {times_same[0]:.2f}ms (API 呼叫)")
print(f"  查詢 2: {times_same[1]:.2f}ms (快取讀取)")
if times_same[1] < times_same[0] * 0.5:
    print(f"  → 加速 {times_same[0]/times_same[1]:.1f}x! ✨")




---

## ? 快速參考：遷移指南

### 原始模式（不推薦）
```python
# ❌ 分離的 server 實例 - 快取無效
async with MCPServerStdio(...) as server:
    result1 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "2317"})

async with MCPServerStdio(...) as server:  # 新進程！
    result2 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "台積電"})

async with MCPServerStdio(...) as server:  # 又是新進程！
    result3 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "0050"})
```

### 改進模式（推薦）
```python
# ✅ 統一的 server 實例 - 快取有效
async with MCPServerStdio(...) as server:
    result1 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "2317"})
    time.sleep(0.3)
    
    result2 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "台積電"})
    time.sleep(0.3)
    
    result3 = await server.session.call_tool("get_taiwan_stock_price", {"symbol": "0050"})
```

### 關鍵差異
| 項目 | 原始模式 | 改進模式 |
|------|--------|--------|
| Server 實例數 | 多個 | 1 個 |
| 進程數 | 多個 | 1 個 |
| 快取有效性 | ❌ 無 | ✅ 有 |
| 響應時間 | ⏱️ 都很長 | ⚡ 第 2+ 次快速 |
| 日誌記錄 | 沒有快取日誌 | 有 `[快取命中]` 日誌 |
| API 呼叫數 | 多次 | 少次 |

---


In [ ]:
# 測試不同的股票代碼（MCP 連接已修復）- 美化 JSON 結果顯示
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "2317"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))


In [ ]:
# 測試：使用公司名稱查詢
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "台積電"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

In [ ]:
# 測試：查詢ETF
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "取得0050股價"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

### 1.2 模擬股票買賣

In [3]:
# 測試：市價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "0050",
            "quantity": 5000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "buy",
      "symbol": "0050",
      "name": "元大台灣卓越50證券投資信託基金",
      "quantity": 5000,
      "price": 62.25,
      "total_amount": 311250.0,
      "current_price": 62.25,
      "order_type": "market",
      "executed": true,
      "message": "市價單立即成交",
      "timestamp": "2025-10-23T10:53:59"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-23T10:54:03.424189",
  "metadata": {
    "status": "success"
  }
}


In [ ]:
# 測試：限價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 2000,
            "price": 500
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

In [ ]:
# 測試：市價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

In [ ]:
# 測試：限價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000,
            "price":550
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

### 1.3 日交易統計

In [ ]:
# 測試：取得日交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_daily_trading",
        {
            "symbol": "2330",
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

## 2. 財務分析工具測試

### 2.1 損益表與資產負債表



### 2.0 統一財務工具測試（單一 Server 實例）

✅ **最佳實踐**：所有財務工具測試在同一 server 實例中執行


In [ ]:
import time

print("=" * 80)
print("【財務工具統一測試 - 使用單一 Server 實例】")
print("=" * 80)

async with MCPServerStdio(params=params, client_session_timeout_seconds=120) as server:
    # 測試 1: 公司基本資訊
    print("\n■ 1. 獲取公司基本資訊 (3231) - 第 1 次")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time1_1 = (time.time() - start) * 1000
    print(f"  耗時: {time1_1:.2f}ms")
    time.sleep(0.3)

    print("■ 1. 獲取公司基本資訊 (3231) - 第 2 次（應從快取讀取）")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    time1_2 = (time.time() - start) * 1000
    print(f"  耗時: {time1_2:.2f}ms ⚡")
    time.sleep(0.3)

    # 測試 2: 損益表
    print("\n■ 2. 獲取損益表 (一般業) - 第 1 次")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "3231"}
    )
    time2_1 = (time.time() - start) * 1000
    print(f"  耗時: {time2_1:.2f}ms")
    time.sleep(0.3)

    print("■ 2. 獲取損益表 (一般業) - 第 2 次（應從快取讀取）")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "3231"}
    )
    time2_2 = (time.time() - start) * 1000
    print(f"  耗時: {time2_2:.2f}ms ⚡")
    time.sleep(0.3)

    # 測試 3: 資產負債表
    print("\n■ 3. 獲取資產負債表 (2330) - 第 1 次")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_balance_sheet",
        {"symbol": "2330"}
    )
    time3_1 = (time.time() - start) * 1000
    print(f"  耗時: {time3_1:.2f}ms")
    time.sleep(0.3)

    print("■ 3. 獲取資產負債表 (2330) - 第 2 次（應從快取讀取）")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_balance_sheet",
        {"symbol": "2330"}
    )
    time3_2 = (time.time() - start) * 1000
    print(f"  耗時: {time3_2:.2f}ms ⚡")
    time.sleep(0.3)

    # 測試 4: 股利資訊
    print("\n■ 4. 獲取股利資訊 (2330) - 第 1 次")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_dividend",
        {"symbol": "2330"}
    )
    time4_1 = (time.time() - start) * 1000
    print(f"  耗時: {time4_1:.2f}ms")
    time.sleep(0.3)

    print("■ 4. 獲取股利資訊 (2330) - 第 2 次（應從快取讀取）")
    start = time.time()
    result = await server.session.call_tool(
        "get_company_dividend",
        {"symbol": "2330"}
    )
    time4_2 = (time.time() - start) * 1000
    print(f"  耗時: {time4_2:.2f}ms ⚡")

print("\n" + "=" * 80)
print("【快取效果統計】")
print("=" * 80)
print(f"公司資訊:    {time1_1:.2f}ms → {time1_2:.2f}ms (加速 {time1_1/time1_2:.1f}x)" if time1_2 > 0 else "")
print(f"損益表:      {time2_1:.2f}ms → {time2_2:.2f}ms (加速 {time2_1/time2_2:.1f}x)" if time2_2 > 0 else "")
print(f"資產負債表:  {time3_1:.2f}ms → {time3_2:.2f}ms (加速 {time3_1/time3_2:.1f}x)" if time3_2 > 0 else "")
print(f"股利資訊:    {time4_1:.2f}ms → {time4_2:.2f}ms (加速 {time4_1/time4_2:.1f}x)" if time4_2 > 0 else "")

print("\n✓ 請查看 logs/mcp.log 檔案查看詳細的快取日誌")


In [ ]:
# 測試：取得綜合損益表 (一般業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "3231"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得綜合損益表 (金融業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "2884"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得資產負債表
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_balance_sheet",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.2 公司基本資訊

In [ ]:
# 測試：取得公司基本資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "3231"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.3 股利資訊

In [ ]:
# 測試：取得股利分配資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_dividend",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得除權息行事曆 (單一公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得除權息行事曆 (所有公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": ""}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.4 營收與估值

In [ ]:
# 測試：取得月營收
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_monthly_revenue", {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得估值比率
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_valuation_ratios", {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 3. 交易統計工具測試

### 3.1 月交易統計

In [ ]:
# 測試：取得月交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_trading",
        {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得月平均價格
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_average",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 3.2 年交易統計

In [ ]:
# 測試：取得年交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_yearly_trading",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 4. 市場資訊工具測試

### 4.1 融資融券

In [ ]:
# 測試：取得融資融券資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_margin_trading_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.2 即時交易統計

In [8]:
# 測試：取得即時交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_real_time_trading_stats",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": false,
  "data": null,
  "error": "No real-time trading stats available",
  "tool": "trading_stats",
  "timestamp": "2025-10-21T17:13:23.865733",
  "metadata": {}
}


### 4.3 ETF排名

In [9]:
# 測試：取得ETF定期定額排名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_etf_regular_investment_ranking",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "ranking_date": null,
    "rankings": [
      {
        "No": "1",
        "STOCKsSecurityCode": "2330",
        "STOCKsName": "台積電",
        "STOCKsNumberofTradingAccounts": "118276",
        "ETFsSecurityCode": "0050",
        "ETFsName": "元大台灣50",
        "ETFsNumberofTradingAccounts": "535855"
      },
      {
        "No": "2",
        "STOCKsSecurityCode": "2886",
        "STOCKsName": "兆豐金",
        "STOCKsNumberofTradingAccounts": "29805",
        "ETFsSecurityCode": "00878",
        "ETFsName": "國泰永續高股息",
        "ETFsNumberofTradingAccounts": "294924"
      },
      {
        "No": "3",
        "STOCKsSecurityCode": "2884",
        "STOCKsName": "玉山金",
        "STOCKsNumberofTradingAccounts": "29751",
        "ETFsSecurityCode": "0056",
        "ETFsName": "元大高股息",
        "ETFsNumberofTradingAccounts": "272193"
      },
      {
        "No": "4",
        "STOCKsSecurityCode": "2317",
        "STOCKsName": "鴻海",
        "STOCKsNumberofTrad

### 4.4 市場指數

In [ ]:
# 測試：取得發行量加權股價指數
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_index_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.5 歷史指數

In [ ]:
# 測試：取得歷史指數資料
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_historical_index",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 5. 外資工具測試

### 5.1 外資持股（按產業）

In [ ]:
# 測試：取得外資各產業持股
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_foreign_investment_by_industry",
        {"count": 8}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 5.2 外資持股前20名

In [ ]:
# 測試：取得外資持股前20名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_top_foreign_holdings",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 6. 節假日工具測試

### 6.1 節假日資訊

In [ ]:
# 測試：查詢元旦
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：查詢中秋節
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-06"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：查詢一般工作日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 6.2 交易日檢查

In [ ]:
# 測試：檢查元旦是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：檢查週六是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-11"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：檢查一般工作日是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))